In [1]:
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
from tqdm.notebook import tqdm
import time
import pandas as pd

Я обрав кухонну техніку, бо саме якраз обирав собі блендер) Я користувався не парсингом їх сторінок, а знайшов запити до їх апі, які робить сама сторінка, щоб завантажити коментарі.

In [2]:
kitchen_url = 'https://bt.rozetka.com.ua/tehnika-dlya-kuhni/c435974/'
page = requests.get(kitchen_url)
soup = BeautifulSoup(page.text)
links = soup.findAll("a", {"class": "portal-navigation__link"})
category_tuples = [(link['href'].split('/')[-3], link['href'].split('/')[-2][1:]) for link in links]
category_tuples

[('coffee_machines', '80164'),
 ('blenders', '80155'),
 ('multivarki', '112986'),
 ('microwaves', '80162'),
 ('grills', '81235'),
 ('electric_kettles', '80160'),
 ('kitchen_machines', '80163'),
 ('meat_choppers', '80176'),
 ('breadmakers', '80154'),
 ('electric_ovens', '80142'),
 ('sandwich_makers', '80180'),
 ('mixers', '80156'),
 ('4625959', '4625959'),
 ('nastolnye-plity', '81667'),
 ('squeezers', '80153'),
 ('toasters', '80145'),
 ('kitchen_scales', '80148'),
 ('fruit_vegetable_dryers', '80697'),
 ('coffee_grinders', '80143'),
 ('yogurt_icecream_machines', '80179'),
 ('airgrills', '81089'),
 ('4625487', '4625487'),
 ('penovzbivateli', '2118637'),
 ('prochaya-melkaya-tehnika', '123281'),
 ('kulery-dlya-vody', '98002'),
 ('steam_cookers', '80157'),
 ('deep_fryers', '80150'),
 ('slicers', '80144'),
 ('blinnitsy', '102312'),
 ('yaytsevarki', '1948582'),
 ('208243', '208243'),
 ('219954', '219954')]

На кожній сторінці категорії міститься по 60 товарів, у кожному товарі коментарі завантажуються по 10. Є сенс дивитись лише на перші кілька сторінок, адже там містяться найбільш популярні товари, що мають хоча б якісь відгуки. Я дивився лише на перші 6 сторінок для кожного типу товарів (але навіть 6 це забагато).

In [9]:
all_comments = []

In [ ]:
for category, category_id in category_tuples:
    print(f'started_category {category}')

    category_url = f'https://xl-catalog-api.rozetka.com.ua/v2/goods/get?category_id={category_id}'
    r = requests.get(category_url)
    ids_metadata = json.loads(r.text)['data']
    pages_in_category = ids_metadata['total_pages']
    print(f'There are {pages_in_category} pages in this category')
    time.sleep(1)
    
    limit_pages = 6 if pages_in_category > 6 else pages_in_category

    for category_page_num in tqdm(range(1, limit_pages)):
        items_page_url = f'https://xl-catalog-api.rozetka.com.ua/v2/goods/get?front-type=xl&category_id={category_id}&page={category_page_num}&sort=rank&lang=ru'
        items_request = requests.get(items_page_url)
        items = json.loads(items_request.text)['data']['ids']
        print(f'[PAGE {category_page_num}/{pages_in_category}] got items list for page {category_page_num}')
        time.sleep(1)
        for item_id in tqdm(items):
            comment_url = f'https://product-api.rozetka.com.ua/v3/comments/get?front-type=xl&goods={item_id}&page={1}&sort=date&limit=10&lang=ru'
            comment_request = requests.get(comment_url)
            comment_json = json.loads(comment_request.text)['data']
            all_comments.extend(comment_json['comments'])

            num_comment_pages = int(np.ceil(comment_json['record']['users_rating']['count_comments'] / 10))
            time.sleep(1)
            for comment_page_num in range(1, num_comment_pages):
                comment_url = f'https://product-api.rozetka.com.ua/v3/comments/get?front-type=xl&goods={item_id}&page={comment_page_num}&sort=date&limit=10&lang=ru'
                comment_request = requests.get(comment_url)
                comment_json = json.loads(comment_request.text)['data']
                all_comments.extend(comment_json['comments'])
                time.sleep(1)
        print(f' After page {category_page_num} we have {len(all_comments)} comments')


In [17]:
import joblib
joblib.dump(all_comments, 'comments_after_all')

['comments_after_all']

Коментарі приходять у вигляді словника, де окрім тексту є якась метаінформація про оцінку товара, кількість апвоутів або усі відповіді на цей відгук. Надалі я брав лише ті відгуки, де виставлена оцінка не 0.

In [21]:
all_comments[5565]

{'id': 34415099,
 'usertitle': 'Владислав Тупчій',
 'mark': 5,
 'percent_dignity': '67',
 'text': 'Дуже достойна кофе ☕ машина... В нас уже 3 роки і працює НОРМАЛЬНО, правда купляли не в розетці, але, РЕКОМЕНДУЮ!!!!',
 'dignity': '',
 'shortcomings': '',
 'from_buyer': False,
 'created': {'pop_date': '9 ноября 2018',
  'year': '2018',
  'month': 11,
  'day': '09'},
 'questions': {'combobox': [], 'marks': [], 'integer': []},
 'attachments': [],
 'lightboxattachmentsjson': '[]',
 'commentdatajson': {'id': 34415099,
  'goods_id': '7444083',
  'user_title': 'Владислав Тупчій',
  'created': '9 ноября 2018',
  'from_buyer': False,
  'mark': 5,
  'text': 'Дуже достойна кофе ☕ машина... В нас уже 3 роки і працює НОРМАЛЬНО, правда купляли не в розетці, але, РЕКОМЕНДУЮ!!!!',
  'dignity': '',
  'shortcomings': '',
  'positive': '2',
  'negative': '1',
  'replies': [],
  'marks': [],
  'questions': [],
  'created_iso': '2018-11-09T22:09:22+02:00'},
 'is_vendor_comment': False,
 'vendor_logo': '',


In [47]:
import spacy
from spacy_langdetect import LanguageDetector

nlp = spacy.load('en_core_web_md')

nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)

nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [35]:
df = pd.DataFrame(all_comments)

df.head()

Розмічаю мову відгуків

In [57]:
texts = df['text'].values

langs = []
for doc in tqdm(nlp.pipe(texts, disable=["ner"])):
    # Do something with the doc here
    langs.append(doc._.language)

lang_labels = [lang['language'] for lang in langs]

df['language'] = lang_labels

In [225]:
df[['text', 'language']].head(10)

,text,language
0,"Долго пользовалась Saeco Royal,но она была оче...",ru
1,Лучшая кофемашина за свою цену.<br />\nВо перв...,ru
2,"Я раньше и не думала что так полюблю кофе, но ...",ru
3,,UNKNOWN
4,,UNKNOWN
5,"Поясніть хтось нарешті, звідки той ""класичний ...",uk
6,"Кофемашина отличная, к ней претензий нет, но е...",ru
7,Хороша машина за свої гроші!,uk
8,Имеет ли данная кофемашина настройку крепости ...,ru
9,Добрый день! Фильтр для воды идёт в комплекте?...,ru


Нажаль, з 77К відгуків маємо лише 14К (з будь якою оцінкою, навіть 0)

In [60]:
df['language'].value_counts()

ru         56450
uk         14268
UNKNOWN     5015
bg           843
mk           559
en             9
ro             7
et             7
so             5
tl             4
sk             3
de             3
tr             2
hr             2
af             2
no             2
cy             1
it             1
fr             1
Name: language, dtype: int64

In [ ]:
uk_df = df[df['language']=='uk'].copy()

uk_df.to_csv('uk_df.csv')

In [101]:
uk_df = uk_df.drop_duplicates(subset='id', keep='first')

Бачимо, що без оцінки дійсно часто лежать питання, тому відкинемо їх. Також, об'єднаю оцінки 1-3 в один "незадовільний клас", адже їх доволі мало.

In [227]:
uk_df['mark'].value_counts()

5.0    5132
0.0    3350
4.0    1254
1.0     407
3.0     367
2.0     256
Name: mark, dtype: int64

In [226]:
uk_df[uk_df['mark']==0.0]['text'].head(15)

5      поясніть хтось нарешті, звідки той "класичний ...
41     питання до представника компанії: який тиск по...
49     відсвяткувавши новосілля, давно мріяли про кав...
65     підкажіть, будь ласка, мінімальні зазори до ме...
85     ця кавомашина ідеальна! та, про яку ми мріяли....
100    в характеристиках вказано, що приготування кап...
122                                 чи є підігрів чашок?
147    підкажіть будь ласка різницю між eq. 6 s100 і ...
167    підскажіть пожалоста при приготуванні лате або...
191    доброго дня. питання щодо піддону під чашками....
243    яку степінь помелу вибрати? і в кого яка стоїт...
253    які жорнова стоять? дві кави одночасно можна г...
255    доброго вам дня, підкажіть будьласка чим відрі...
271    нещодавно придбав philips series 3200 ep3243/5...
272    нещодавно придбав для офісу. кава готується пр...
Name: text, dtype: object

Дивимось на приклади з різнии оцінками

In [236]:
for mark in range(1,6):
    print(f'MARK {mark}')
    for i in range(5):    
        print(uk_df[uk_df['mark']==mark]['text'].reset_index().loc[i]['text'])
        print(' ')
    print('----------------------------------')

MARK 1
кавомашина виявилась несправна, при першому включенні видало помилка 11. дуже сподівався, що це окліматизація, але помилка так і не зникла. в інтернеті знайшов коди помилок і пишуть, що це несправність датчика паронагрівача. затра повезу повертати. шкода потраченого часу.
 
поробила всього 3 місяці і здохла.
 
купили машину в сентябре, гарантийного талона в посылке не было. не потребовали сразу. а зря. в феврале сломалась мельница. в сервисном центре требуют гарантийный талон. написала письмо в поддержку, чтобы прислали талон, предоставила все доки( чек, номер заказа, фото серийного номера ) .знаете что мне ответили?  "зараз нам надходить дуже велика кількість запитів, тому, на жаль, термін очікування відповіді може виявитись більшим, ніж нам би того хотілось.". то есть, люди плохо сделали свою работу, и говорят мне "подождите, нам некогда"???
 
придбали кавоварку 22.11.2019р.<br />
не збиває пінку з молока,не робить лате.
 
начитавшись позитивних відгуків вирішили придбати дани

In [88]:
joblib.dump(uk_df, 'uk_df.joblib')

['uk_df.joblib']

In [103]:
uk_df['mark_class'] = uk_df['mark'].map(
    {
        5.0: 'positive',
        4.0: 'neutral',
        3.0: 'negative',
        2.0: 'negative',
        1.0: 'negative'
    }
)

In [104]:
uk_df['mark_class'].value_counts()

positive    5132
neutral     1254
negative    1030
Name: mark_class, dtype: int64

In [114]:
uk_df['text'] = uk_df['text'].str.lower()

###  Моделювання

In [171]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

In [124]:
X_train, X_test, y_train, y_test = train_test_split(
    uk_df[uk_df['mark_class'].notnull()]['text'], 
    uk_df[uk_df['mark_class'].notnull()]['mark_class'],
    test_size=0.2,
    random_state=42
)

Спочатку подивимось на метрики, якщо передбачувати лише позитивну оцінку

In [139]:
y_pred_very_baseline = ['positive']*len(y_test)

In [140]:
print(classification_report(y_test, y_pred_very_baseline))

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00       208
     neutral       0.00      0.00      0.00       245
    positive       0.69      1.00      0.82      1031

    accuracy                           0.69      1484
   macro avg       0.23      0.33      0.27      1484
weighted avg       0.48      0.69      0.57      1484



/home/yevhen/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## 1. Будуємо базове рішення на звичайному векторайзері

In [256]:
count_lr_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', LogisticRegression(
        random_state=42, 
        solver = 'sag',
        multi_class='multinomial',
        max_iter=6000,
    )),
])

In [262]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score

In [290]:
def print_report_for_cv(X, y, clf, cv):
    originalclass = []
    predictedclass = []
    def classification_report_with_accuracy_score(y_true, y_pred):
        originalclass.extend(y_true)
        predictedclass.extend(y_pred)
        return accuracy_score(y_true, y_pred)

    nested_score = cross_val_score(
        count_lr_clf, 
        X=X, 
        y=y, 
        cv=cv, 
        scoring=make_scorer(classification_report_with_accuracy_score)
    )
    print(classification_report(originalclass, predictedclass)) 

In [291]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

print_report_for_cv(X_train, y_train, count_lr_clf, cv)

              precision    recall  f1-score   support

    negative       0.67      0.46      0.55       822
     neutral       0.38      0.23      0.29      1009
    positive       0.79      0.91      0.85      4101

    accuracy                           0.74      5932
   macro avg       0.61      0.54      0.56      5932
weighted avg       0.70      0.74      0.71      5932



## 2. Зроблю те саме, але на лемах

In [149]:
import stanza

stanza_nlp = stanza.Pipeline('uk')

In [158]:
def get_lemmatized_review(review):
    doc = stanza_nlp(review)
    lemmas = [word.lemma for word in doc.iter_words()]
    return ' '.join(lemmas)

In [160]:
X_train_lemmas = [get_lemmatized_review(r) for r in X_train]
X_test_lemmas = [get_lemmatized_review(r) for r in X_test]

In [162]:
# joblib.dump(X_train_lemmas, 'X_train_lemmas')
# joblib.dump(X_test_lemmas, 'X_test_lemmas')

['X_test_lemmas']

In [292]:
print_report_for_cv(X_train_lemmas, y_train, count_lr_clf, cv)

              precision    recall  f1-score   support

    negative       0.63      0.49      0.55       822
     neutral       0.35      0.21      0.26      1009
    positive       0.79      0.91      0.85      4101

    accuracy                           0.73      5932
   macro avg       0.59      0.54      0.55      5932
weighted avg       0.69      0.73      0.71      5932



In [280]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

## 3. TF-IDF + Lemmas

In [285]:
tfidf_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(
        random_state=42, 
        solver = 'sag',
        multi_class='multinomial',
        max_iter=6000,
    )),
])

In [293]:
print_report_for_cv(X_train, y_train, tfidf_clf, cv)

              precision    recall  f1-score   support

    negative       0.67      0.46      0.55       822
     neutral       0.38      0.23      0.29      1009
    positive       0.79      0.91      0.85      4101

    accuracy                           0.74      5932
   macro avg       0.61      0.54      0.56      5932
weighted avg       0.70      0.74      0.71      5932



In [294]:
print_report_for_cv(X_train_lemmas, y_train, tfidf_clf, cv)

              precision    recall  f1-score   support

    negative       0.63      0.49      0.55       822
     neutral       0.35      0.21      0.26      1009
    positive       0.79      0.91      0.85      4101

    accuracy                           0.73      5932
   macro avg       0.59      0.54      0.55      5932
weighted avg       0.69      0.73      0.71      5932



## 4. Біграми на лемах

Подивимось, які біграми з лем вдається витягти по класах

In [300]:
#got function here https://medium.com/@cristhianboujon/how-to-list-the-most-common-words-from-text-corpus-using-scikit-learn-dad4d0cab41d
def get_top_n_words(corpus, n=None, ngrams=2):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vec = CountVectorizer(
        ngram_range=(ngrams,ngrams),
    ).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]


lemma_df = pd.DataFrame({
    'rev': X_train_lemmas, 
    'label': y_train
})

for ngram in (2,3):
    print(f' NGRAM = {ngram}')
    for label in ('negative','neutral', 'positive'):
        print(f'LABEL IS {label}')
        label_revs = lemma_df[lemma_df['label']==label]['rev'].values
        print(get_top_n_words(label_revs, 80, ngrams=2))
        print(' ')

 NGRAM = 2
LABEL IS negative
[('не рекомендувати', 55), ('не бути', 49), ('сервісний центр', 46), ('гарантійний талон', 45), ('те що', 44), ('не працювати', 38), ('це не', 33), ('так як', 32), ('за такий', 28), ('сказати що', 26), ('не могти', 26), ('що не', 25), ('не дуже', 24), ('br br', 24), ('ніщо не', 22), ('не знати', 22), ('перестати працювати', 21), ('що це', 20), ('не відповідати', 20), ('тому що', 20), ('по гарантія', 19), ('але не', 19), ('він не', 19), ('під час', 19), ('талон не', 19), ('при перший', 19), ('на розетець', 18), ('не мати', 18), ('через місяць', 17), ('вона не', 17), ('перший використання', 17), ('не заповнений', 17), ('свій гроші', 16), ('думати що', 16), ('br не', 16), ('новий пошта', 16), ('такий гроші', 15), ('взагалі не', 15), ('не радити', 14), ('але це', 14), ('на день', 14), ('так не', 14), ('запах пластика', 13), ('що бути', 13), ('після перший', 13), ('не задоволений', 13), ('це бути', 13), ('не можна', 13), ('на новий', 13), ('на подарунок', 13), (

Бачимо, що для для відгуків різних класів іноді характерні усілякі сталі вирази, що погано або гарно характеризують товар: 
* біграми ('не рекомендувати' або 'дуже задоволений')
* триграми ('викинутий на вітер' або 'помічник на кухні'). 
Водночас, 'за такі гроші' є в усіх трьох класах, тобто за свої гроші може бути, як погане, так і хороше)

Начебто біграми та триграми на словах мають гарно впливати на класифікацію сентименту

In [301]:
count_lr_wbi_clf = Pipeline([
    ('vect', CountVectorizer(
        ngram_range=(2,3),
    )),
    ('clf', LogisticRegression(
        random_state=42, 
        solver = 'sag',
        multi_class='multinomial',
        max_iter=6000,
    )),
])

In [295]:
print_report_for_cv(X_train_lemmas, y_train, count_lr_wbi_clf, cv)

              precision    recall  f1-score   support

    negative       0.63      0.49      0.55       822
     neutral       0.35      0.21      0.26      1009
    positive       0.79      0.91      0.85      4101

    accuracy                           0.73      5932
   macro avg       0.59      0.54      0.55      5932
weighted avg       0.69      0.73      0.71      5932



In [303]:
import numpy as np

def top_features(vectorizer, clf, n):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer.get_feature_names()
    for i, class_label in enumerate(clf.classes_):
        top = np.argsort(clf.coef_[i])
        reversed_top = top[::-1]
        print("%s: %s\n" % (class_label,
              " ".join(feature_names[j] for j in reversed_top[:n])))


In [315]:
vectorizer = CountVectorizer(ngram_range=(2,3)).fit(X_train_lemmas)
lr =LogisticRegression(
        random_state=42, 
        solver = 'sag',
        multi_class='multinomial',
        max_iter=6000,
    )
lr.fit(vectorizer.transform(X_train_lemmas), y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=6000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [316]:
top_features(vectorizer, lr, 50)

negative: не рекомендувати не відповідати не працювати через місяць працювати задоволений сервісний центр що робити через день інший магазин ну такоє іржа наступати не беріти таки працювати перестати працювати взагалі задоволений поки немати два місяць через тиждень після місяць перестати включатися відсутній інструкція неприємний запах для дерун після перший br не свій грошї за свій грошї гроші можна не брати чудово працювати задоволений прийти без такий якість не розігрівати цен нормально такуй цен нормально при перший для дім згодитися дім згодитися що причина по гарантія такий гроші нормальний на новий не закриватися вигляд мати гарний місяць використання за хороший сервіс розетець за хороший перший використання микрохвильовий піч вигляд мати

neutral: ціле задоволений очень понравитися претензія немати якість ціна рекомендувати дякувати по акція бути користуватися нормальний якість не поганий робити рік все простенько загальний позитивний вчитися користуватися трішки шумити можно 

Бачимо,що моделі вдається вивчити, які бі-, триграми характерні для позитивних та негативних відгуків. Водночас, результати порівняні або навіть трохи гірше від базового рішення